In [57]:
# Import libraries and utils
%run "utils_for_doubao.ipynb"

select interesting prompting conditions (drop some unnecessary ones) F1-zero shot, F3-few shot, F5-categorical scale, F8-conversational, F9-single word pair

sampling from multisimlex instead of doing all of them - noun 1051, verb 461, adjective 245, adverb 123; sample 100 from each category? The original distribution seems not too important in this research

experimental design, work out a rough workflow

something similar to a research proposal

introduction, background lit (also Chinese context - model, benchmark, semantic similarity "why Chinese is interesting", what tasks have been done, what gap compared to English 

simlex research, linguistic and interpretability "who cited multisimlex and are they related to Chinese") and methodology (models, dataset, experiment)

In [15]:
sampled_avg_df = pd.read_csv('sampled_avg_multisimlex_CN.csv')
sampled_avg_df

,Unnamed: 0,ID,Word 1,Word 2,PoS,Annotator 1,Annotator 2,Annotator 3,Annotator 4,Annotator 5,Annotator 6,Annotator 7,Annotator 8,Annotator 9,Annotator 10,Annotator 11,avg_anno
0,1073,1074,长,狭窄,adjectives,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.27
1,1062,1063,内疚,羞愧,adjectives,2.0,6.0,6.0,5.0,4.0,5.0,3.0,3.0,5.0,4.0,5.0,4.36
2,1138,1139,暴力,被动,adjectives,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,1064,1065,闷,平庸,adjectives,0.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,1.0,2.0,0.91
4,1171,1172,害羞,昂贵,adjectives,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1462,1463,离开,走开,verbs,5.0,3.0,5.0,5.0,3.0,5.0,3.0,6.0,5.0,2.0,3.0,4.09
468,1706,1707,读,理解,verbs,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.64
469,1700,1701,装作,想像,verbs,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.18
470,1318,1319,跟随,追逐,verbs,2.0,1.0,4.0,3.0,1.0,2.0,3.0,6.0,4.0,4.0,1.0,2.82


In [21]:
tuples_list = list(zip(sampled_avg_df['Word 1'], sampled_avg_df['Word 2']))

In [23]:
prompt_en = ("Rate the semantic similarity of each word pair on a scale from 0 to 6, "
          "where 0 represents no semantic similarity, and 6 represents perfect semantic similarity. "
          "Use only intergers. The response should strictly adhere to the structure: "
          "[('word1', 'word2', <score>), ('word3', 'word4', <score>), ...]. "
          "Do not provide additional explanations or context.")
prompt_cn = ("请给每组词语的语义相似性按照从0到6的刻度打分，"
             "其中0代表语义完全不相似，6代表语义完全一致。"
             "所给分数仅保留整数。回答应该严格按照如下格式："
             "[('词语1'，'词语2'，<分数>)，('词语3'，'词语4'，<分数>)，...]。"
             "请勿提供任何多余的解释或上下文。")

In [25]:
chunks_of_pairs = split_into_n_lists(tuples_list, 20)
print_prompts(chunks_of_pairs, prompt_cn)

请给每组词语的语义相似性按照从0到6的刻度打分，其中0代表语义完全不相似，6代表语义完全一致。所给分数仅保留整数。回答应该严格按照如下格式：[('词语1'，'词语2'，<分数>)，('词语3'，'词语4'，<分数>)，...]。请勿提供任何多余的解释或上下文。 --- ["('长', '狭窄'), ('内疚', '羞愧'), ('暴力', '被动'), ('闷', '平庸'), ('害羞', '昂贵'), ('不同', '正常'), ('自豪', '傲慢'), ('愉快', '快乐'), ('粗糙', '无礼'), ('坚硬', '密度大'), ('无实体', '心灵'), ('开心', '高兴'), ('开心', '疯狂'), ('合法', '允许'), ('和蔼', '残酷'), ('胆大', '自豪'), ('坏', '平庸'), ('煮熟的', '水煮'), ('残忍', '不公平'), ('不需要', '必须'), ('沈闷', '枯燥'), ('自由', '无辜'), ('善良', '大方'), ('弱', '辣')"]
请给每组词语的语义相似性按照从0到6的刻度打分，其中0代表语义完全不相似，6代表语义完全一致。所给分数仅保留整数。回答应该严格按照如下格式：[('词语1'，'词语2'，<分数>)，('词语3'，'词语4'，<分数>)，...]。请勿提供任何多余的解释或上下文。 --- ["('兴奋', '受惊'), ('无知', '频繁'), ('怪诞', '奇怪'), ('人造', '假'), ('电动', '甜'), ('婚外', '不可說出口的'), ('笨', '傻'), ('个人', '私人'), ('瘦', '苗条'), ('奇怪', '古怪'), ('灵活', '松动'), ('强大', '强大'), ('大', '柔韧'), ('脆弱', '简单'), ('粘人', '缺乏信心'), ('显著的', '明显'), ('暴力', '柔软'), ('害怕', '急'), ('非法', '不道德'), ('严酷', '残酷'), ('粗糙', '丑陋'), ('难以置信', '普通'), ('尖', '平滑'), ('容易', '灵活')"]
请给每组词语的语义相似性按照从0到6的刻度打分，其中0代表语义完全不相似，6代表语义完全一致。所给分

In [ ]:
import os
os.environ['ARK_API_KEY'] = ''

In [49]:
# gets API Key from environment variable OPENAI_API_KEY
# Get api key
load_dotenv()
OPENAI_API_KEY = os.getenv('ARK_API_KEY')
client = OpenAI(
    api_key=os.environ.get("ARK_API_KEY"),
    base_url="https://ark.cn-beijing.volces.com/api/v3",
)

In [52]:
# Define model
model = "ep-20241006053554-cz7vx"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 15.0

# Define number of sublists
n_sublists = 20

In [63]:
response = get_responses(chunks_of_pairs, prompt_cn, model, sample_size, delay)

Processing: 100%|███████████████████████████████████████████████████████████████| 300/300 [2:11:35<00:00, 26.32s/chunk]

Total time taken: 7895.53 seconds


In [67]:
# Define filepath
file_path = './doubao/f1.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [70]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{}


In [74]:
combined_multisimlex = sampled_avg_df.copy()
combined_multisimlex

,Unnamed: 0,ID,Word 1,Word 2,PoS,Annotator 1,Annotator 2,Annotator 3,Annotator 4,Annotator 5,Annotator 6,Annotator 7,Annotator 8,Annotator 9,Annotator 10,Annotator 11,avg_anno
0,1073,1074,长,狭窄,adjectives,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.27
1,1062,1063,内疚,羞愧,adjectives,2.0,6.0,6.0,5.0,4.0,5.0,3.0,3.0,5.0,4.0,5.0,4.36
2,1138,1139,暴力,被动,adjectives,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,1064,1065,闷,平庸,adjectives,0.0,0.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,1.0,2.0,0.91
4,1171,1172,害羞,昂贵,adjectives,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1462,1463,离开,走开,verbs,5.0,3.0,5.0,5.0,3.0,5.0,3.0,6.0,5.0,2.0,3.0,4.09
468,1706,1707,读,理解,verbs,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,1.0,0.64
469,1700,1701,装作,想像,verbs,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.18
470,1318,1319,跟随,追逐,verbs,2.0,1.0,4.0,3.0,1.0,2.0,3.0,6.0,4.0,4.0,1.0,2.82


In [77]:
# Print duplicate word pairs
print_duplicate_word_pairs(sampled_avg_df, data_dict)
# This missing pair detected is due to an orthographical difference of the same character, ignored

    Combined_Columns
384            离开_參加
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [80]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,长,狭窄,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,内疚,羞愧,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
2,暴力,被动,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,闷,平庸,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,害羞,昂贵,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,离开,走开,5.0,5.0,6.0,6.0,5.0,5.0,6.0,6.0,5.0,5.0,6.0,5.0,5.0,5.0,6.0
468,读,理解,3.0,3.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,3.0,3.0,2.0
469,装作,想像,3.0,3.0,2.0,2.0,3.0,3.0,2.0,4.0,2.0,3.0,4.0,3.0,3.0,3.0,2.0
470,跟随,追逐,3.0,3.0,2.0,4.0,3.0,3.0,2.0,4.0,2.0,3.0,4.0,3.0,3.0,3.0,2.0


In [84]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    0
dtype: int64


In [89]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


In [99]:
# Define file_path
file_path = 'doubao/f1.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
